# Lab 1 - Parte 2 - Ranking e Modelo Vetorial

Nesta atividade, foi feita a implementação de alguns algoritmos de busca usando o modelo vetorial. O índice invertido da primeira parte da atividade foi reaproveitado e melhorado para que este fim fosse atingido.

Primeiro, foram importadas as bibliotecas necessárias:

In [1]:
import csv
import timeit
import operator
import math
import random
import ast
from unidecode import unidecode
from sortedcontainers import SortedList
from collections import Counter

E definida uma função para regularizar o texto, tirando todos os caracteres especiais e acentuações que fossem encontrados nestes:

In [2]:
def standardize_text(text):
    return unidecode(''.join(e for e in text if (e.isalnum() or e ==' ')).lower())

## Índice invertido

Iniciam-se as variáveis que ajudarão na definição do ínidice invertido.

In [3]:
inverted_index = dict()
page_tf = dict()
general_tf = dict()

Para então definir as funções necessárias para o funcionamento do índice invertido: 

In [4]:
def fill(page_id, text):
    word_counts = Counter(text.split(' '))
    page_tf[page_id] = word_counts

    for word in word_counts:
        if word not in general_tf:
            general_tf[word] = 0

        general_tf[word] += word_counts[word]

        if word not in inverted_index:
            inverted_index[word] = SortedList()

        if page_id not in inverted_index[word]:
            inverted_index[word].add(page_id)

_fill_ é responsável por encher o índice invertido através de um texto recebido

In [5]:
def get_postings(term):
    return inverted_index[term]


def get_tf(term, page_id):
    return page_tf[page_id][term]


def get_general_tf(term):
    return general_tf[term]

*get_postings*, *get_tf* e *get_general_tf* retornam a lista de documentos em um termo é encontrado, a frequência de um termo em um documento e a frequência de um termo em todos os documentos, respectivamente.

In [6]:
def calculate_idf(word):
    return math.log((len(inverted_index) + 1)/len(inverted_index[word]))

def calculate_bm25(word,page):
    k = random.random() * 0.8 + 1.2

    return calculate_idf(word) * ((get_tf(word, page) * (k + 1))/(get_tf(word, page) + k))

Aqui foram definidas as duas funções que realizam os cálculos que serão necessários no modelo, uma de IDF, que segue a seguinte fórmula:

$$IDF = log\frac{M + 1}{k}$$

Em que M é o número de termos no índice invertido e k é o número de documentos que o termo é encontrado. 
A outra função define o cálculo de score BM25, que segue a fórmula:

$$score = IDF * \frac{f(t,D) * (k + 1)}{f(t,D) + k}$$

Em que f(t,D) é a frequência de um termo em um documento e k é uma constante com valor aleatório entre 1.2 e 2

Pode-se então agora, finalmente, definir as funções de busca.

In [7]:
def search(sentence, method):
    standardized_sentence = standardize_text(sentence)
    words = standardized_sentence.split(' ')

    return {
        'tf': score_search(words, 'tf'),
        'tfidf': score_search(words, 'idf'),
        'bm25': score_search(words, 'bm25'),
        'binary': binary_search(words)
    }[method]

*search* define qual dos 4 métodos usar, sendo que o **binary** já tinha sido implementado na atividade passada. Revisando o algoritmo usado para este, tem-se:

In [8]:
def binary_search(word_list):
    if len(word_list) == 1:
        return get_postings(word_list[0])

    results = None

    for i in range(1, len(word_list)):
        if i == 1:
            results = merge_results(get_postings(word_list[0]), get_postings(word_list[1]))

        else:
            results = merge_results(results, get_postings(word_list[i]))

    return results

def merge_results(first_list, second_list):
    results = SortedList()

    i = 0
    j = 0

    while True:
        if first_list[i] == second_list[j]:
            results.add(first_list[i])

        if first_list[i] > second_list[j]:
            j += 1
            if j == len(second_list):
                break

        else:
            i += 1
            if i == len(first_list):
                break

    return results

Pode-se então, por fim, definir as buscas rankeadas com a função *score_search*

In [9]:
def score_search(word_list, model):
    results = binary_search(word_list)

    sorted_results = []

    for page in results:
        total_score = 0

        for word in word_list:
            if model == 'bm25':
                score = calculate_bm25(word, page)
            else:
                score = get_tf(word, page) * (1 if model == 'tf' else calculate_idf(word))

            total_score += score

        sorted_results.append({'id': page, 'score': total_score})

    sorted_results.sort(key=operator.itemgetter('score'), reverse=True)
    return [e['id'] for e in sorted_results]

A função segue a mesma estrutura para os três métodos (bm25, tf e tf-idf) a única diferença é na hora de calcular o score para cada sentença de busca. No bm25 usa-se a função de cálculo do score de bm25, no tf pega-se a frequência de cada termo no documento e no tf-idf usa-se a frequência de termo no documento e multipla-se pelo idf de uma palavra.

Com isso, o índice invertido fica definido.

## Fazendo a busca

Para fazer a busca, importam-se os dados e enche-se o índice invertido:

In [10]:
def join_list(article_sections):
    return ' '.join(str(e) for e in article_sections)

ID_INDEX = 5
with open('estadao_noticias_eleicao.csv', 'rt', encoding='utf8') as csvfile:
    estadao = csv.reader(csvfile)

    for index, text_row in enumerate(estadao):
        if index == 0:
            continue

        pageId = int(text_row[ID_INDEX])
        content = standardize_text(join_list(text_row[1:4]))

        fill(pageId, content)

O sub-array definido das posições de 1 a 4 e a função *join_list* servem para juntar título, sub-título e conteúdo do documento em uma só string, dessa forma torna-se mais fácil o preenchimento do índice invertido.

Agora é possível fazer a busca de termos usando a função *search* e escolhendo um dos métodos. Por exemplo, pode ser feito a busca: 'lula bolsonaro'. Fazendo *search('lula bolsonaro', 'tf')*

In [11]:
print(search('lula bolsonaro', 'tf'))

[932, 43, 46, 2187]


Vê-se portanto, que quatro documentos tem os termos lula **E** bolsonaro e desses, o mais relevante, pelo método **tf**, é o documento 932

In [12]:
print(search('lula bolsonaro', 'bm25'))

[932, 2187, 46, 43]


Mudando de método vê-se que o 2187 que era o último em relevância, passou a ser segundo.

## Corretude

A parte final da atividade consistia em definição de corretude de acordo com um gabarito fornecido. Para auxiliar nessa parte, foi usada função pronta *MAPK*:

In [13]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])


Importa-se, então, o gabarito:

In [14]:
SEARCH_SENTENCE_INDEX = 0
GOOGLE_ANSWER_INDEX = 1
TF_ANSWER_INDEX = 3
TF_IDF_ANSWER_INDEX = 4
BM25_ANSWER_INDEX = 5

search_sentences =[]
correct_answer_sheet = {'google' : [], 'tf': [], 'tfidf': [], 'bm25': []}

with open('gabarito.csv', 'rt', encoding='utf8') as csvfile:
    gabarito = csv.reader(csvfile)
   
    for index, row in enumerate(gabarito):
        if index == 0:
            continue
        
        search_sentences.append(row[SEARCH_SENTENCE_INDEX])
        correct_answer_sheet['google'].append(ast.literal_eval(row[GOOGLE_ANSWER_INDEX]))
        correct_answer_sheet['tf'].append(ast.literal_eval(row[TF_ANSWER_INDEX]))
        correct_answer_sheet['tfidf'].append(ast.literal_eval(row[TF_IDF_ANSWER_INDEX]))
        correct_answer_sheet['bm25'].append(ast.literal_eval(row[BM25_ANSWER_INDEX]))

E agora, usando os mesmos termos, cria-se a resposta da atividade usando os modelos implementados:

In [15]:
our_answer_sheet = {'tf': [], 'tfidf': [], 'bm25': []}
for search_sentence in search_sentences:
    for model in our_answer_sheet:
        our_answer_sheet[model].append(search(search_sentence, model)[0:5])

Finalmente, calcula-se a a corretude:

In [16]:
accuracy = {}
for model in our_answer_sheet:
    accuracy[model] = mapk(correct_answer_sheet[model], our_answer_sheet[model],5)

In [17]:
print(accuracy)

{'tf': 0.9199999999999999, 'tfidf': 0.5453333333333333, 'bm25': 0.41}


Por fim, comparando com a do google:

In [18]:
google_accuracy = {}
for model in our_answer_sheet:
    google_accuracy[model] = mapk(correct_answer_sheet['google'], our_answer_sheet[model],5)

In [19]:
print(google_accuracy)

{'tf': 0.048, 'tfidf': 0.048, 'bm25': 0.12000000000000002}


## Conclusões

Viu-se que, por mais que o acerto tenha sido baixo, o bm25 foi o modelo que mais se aproximou da resposta dada pelo Google. Isso porque este é o único dos modelos que tenta usar normalização para que uma palavra que é considerada relevante, porém muito repetida, não garanta, necessariamente, um score muito alto para um documento.   
Além disso, comparando o gabarito e as respostas obtidas, viu-se que o maior acerto foi o do modelo tf e o menor foi o modelo bm25, isso era esperado porque o modelo bm25 usa uma variável aleatória, como a lista de acerto era pequena então se os valores de *k* fossem muito diferentes poderia afetar bastante a resposta. Os modelos *idf* e *tf* estarem com acurácia diferente de 100% causa estranheza, mas acredita-se que isto ocorreu porque o algoritmo de busca binária conjuntiva, da primeira atividade, foi reaproveitado, descartando então documentos em que não houvessem todas as palavras da pesquisa.